# Tehnici de Compilare: Tema 2 Problema 2 
## Dilirici Radu (332.1)
## Surcea Mihai-Daniel (332.1)

### Cerinta:
* Sa se scrie un program care primeste la intrare elementele unei gramatici independente de context oarecare, G, inclusiv cu 𝜆-productii pentru care:
    * calculeaza multimile First(X), Follow(X), pentru fiecare simbol X terminal sau neterminal
    * elimina recursivitatea la stanga (pentru gramatici fără 𝜆-producții)
    * factorizează stânga gramatica G.

In [1]:
import numpy as np
from pprint import pprint
from string import ascii_letters

def MaxPrefix(v, step, mx):
    # scrisa cu ajutorul https://www.geeksforgeeks.org/longest-string-which-is-prefix-string-of-at-least-two-strings/
    prefix = ''

    for c in ascii_letters:
        v1 = []
        
        for j in range(len(v)):
            if len(v[j]) > step:
                if v[j][step] == c:
                    v1.append(v[j])
            
        if len(v1) >= 2:
            mx_aux, prefix_aux = MaxPrefix(v1, step + 1, mx)
            if mx_aux > mx:
                mx = mx_aux
                prefix = prefix_aux
        else:
            if step > mx:
                mx = step
                prefix = v[0][:step]
            
    return mx, prefix

class GIC: # Clasa Gamaticilor Independente de Context
    LAMBDA  = ''
    LAMBDAF = '#'
    VerifyDefault=True

    def __init__(self, N, Sigma, P, S='S', isLambda=True, verify=VerifyDefault):

        self.N = N               # Alfabetul Neterminalilor
        self.T = Sigma           # Alfabetul   Terminalelor
        self.S = S               # Simbolul de start
        self.P = P.copy()        # Multimea de Productii
        self.isLambda = isLambda # Daca acest GIC este accepta 

        self.updateElements()
        
        if verify: # validarea 
            assert self.S in self.N, "Simbolul de start nu este Neterminal"
            _, ind = np.unique(self.N, return_counts=True)
            assert (ind == 1).all(), "Alfabetul Neterminalilor nu este o multime"
            _, ind = np.unique(self.N, return_counts=True)
            assert (ind == 1).all(), "Alfabetul Terminalilor nu este o multime"
            assert len(np.intersect1d(self.N, self.T, assume_unique=True)) == 0, "Intersectia Alfabetului Neterminalilor si Terminalilor nu este vid"

            for key, val in self.P.items():
                assert key in self.N and np.isin(np.concatenate(val), self.elems).all(), 'Multimea de productii nu este valida'

    def updateElements(self):
        self.elems = np.concatenate((self.N, self.T, [self.LAMBDA, self.LAMBDAF]) if self.isLambda else (self.N, self.T, [self.LAMBDAF]))
        return self

    def _FirstRec(self, k, word, curr=[], ret=[], toPrint=False):

        if len(word) < 1 or k == 0: 
            if toPrint: print('-', ''.join(curr))
            return ret + [curr]

        if word[0] == self.LAMBDA:
            return self._FirstRec(k, word[1:], curr+[self.LAMBDA], ret, toPrint) # ignoram simbolul LAMBDA

        newV = self.P.get(word[0], [])

        if len(newV) < 1: # Terminal
            return self._FirstRec(k-1, word[1:], curr+[word[0]], ret, toPrint)

        # Neterminal
        words = [v + word[1:] for v in newV]
        output = []
        if toPrint: print(f'Frs {k}', ''.join(curr+word), list(map(lambda x: ''.join(curr+x), words)))
        for w in words:
            ret = self._FirstRec(k,w,curr,ret, toPrint)
        return ret

    def First(self, k, word, toPrint=False, verify=VerifyDefault, Following=False):
        if k == 0: return [self.LAMBDA]
        assert k > 0, 'Nu poti avea numar negativ de litere'
        if verify:
            assert np.isin(word, self.elems).all(), f'Nu toate literele cuvantului sunt din vocabular: {word}'
        
        output = self._FirstRec(k, word, toPrint=toPrint)

        # if Following: return output

        output = list(map(lambda x: ''.join(x), output))

        if toPrint: print('\n', output, sep='')

        return list(np.unique(output)) if not Following else output

    
    def FirstAll(self, k=1, toPrint=False, verify=VerifyDefault):
        d = {}
        for n in self.N:
            d[n] = self.First(k=k, word=[n], toPrint=toPrint, verify=verify)
        return d

    def FollowAll(self, k=1):
        Follow = {}
        NBool = {self.LAMBDA: False, self.LAMBDAF:False}
        for n in self.N:
            Follow[n] = []
            NBool[n] = True
        for n in self.T:
            NBool[n] = False
        Follow['S'] = [self.LAMBDAF]
        stillAdding = True

        iteration = 0
        while True:
            iteration += 1
            stillAdding = False
            for A, allProds in self.P.items():
                for prod in allProds:
                    for idx, B in enumerate(prod):
                        if NBool[B]:
                            oldFollowB = Follow[B]
                            toAddFollowB = []
                            for w in Follow[A]:
                                word = prod[idx+1:] + list(w)
                                rez = self.First(k=k, word=word, Following=True, verify=True)
                                toAddFollowB.extend(rez)
                            if len(toAddFollowB) < 1: continue
                            newFollowB = np.union1d(oldFollowB, toAddFollowB)
                            if (len(newFollowB) > len(oldFollowB)):
                                Follow[B] = newFollowB
                                stillAdding = True
            
            if not stillAdding: break

            print('Iteration', iteration)
            pprint(Follow)       
            print('')


        # return Follow
    
    def ElimRecStIm(self, A):  # ELIMINAREA RECURSIVITĂȚII LA STÂNGA IMEDIATE
        A_ = A + "'"
        tr_A = []  # noile productii de la A
        tr_A_ = []  # viitoarele productii de la A'
        productii = self.P.get(A)
        for productie in productii:
            if productie[0] == A:  # caz de recursivitate imediata
                tr_A.append(productie[1:] + [A_])
            else:
                tr_A_.append(productie + [A_])
        tr_A_.append([LAMBDA])

        if len(tr_A) < 1 or len(tr_A_) < 1:
            return

        self.P[A] = tr_A
        self.P[A_] = tr_A_
        self.N.append(A_)

    def ElimRecSt(self):  # ELIMINAREA RECURSIVITĂȚII LA STÂNGA
        N = list(self.P.keys())
        for Ai in N:
            for Aj in N:
                if Aj == Ai:
                    break

                productii = self.P[Ai]
                for productie in productii:
                    if productie[0] == Aj:
                        self.P[Ai] = [[delta] + productie[1:] for delta in self.P[Aj]]
            
            self.ElimRecStIm(Ai)
        self.updateElements()
    
    def FactSt(self):
        N = list(self.P.keys())
        i = -1
        while True:
            i += 1
            if i >= len(N):
                break
            
            A = N[i]

            while True:
                productii = self.P[A]
                mx, prefix = MaxPrefix(productii, 0, 0)
                if mx < 1:
                    break

                gamma = []  # productiile care nu au prefixul gasit

                A_ = A + "'"  # gasim urmatoarea notatie disponibila
                while True:
                    if A_ not in self.P:
                        break
                    A_ = A_ + "'"

                prod_A_ = []
                for productie in productii:
                    if productie[:mx] == prefix:
                        prod_A_.append(productie[mx:] or [''])
                    else:
                        gamma.append(productie)
                
                self.P[A] = [prefix + [A_]] + gamma
                self.P[A_] = prod_A_
                self.N.append(A_)
                N.append(A_)
        self.updateElements()

In [2]:
# Seminar 4: Ex1. a)

# Se da gramatica:
LAMBDA = GIC.LAMBDA
N = ['S', 'A']
T = ['a', 'b', 'c']
P = {
    'S': [['a', 'S', 'b', 'S'], ['a', 'A']],
    'A': [['b', 'A', 'c'], [LAMBDA]]
}
G = GIC (N, T, P)

# Sa se calculeze First3 (AaS)
G.First(k=3, word=['A', 'a', 'S'])

['aa', 'aaa', 'aab', 'bbb', 'bbc', 'bca']

In [3]:
# Example First: https://www.geeksforgeeks.org/first-set-in-syntax-analysis/

LAMBDA = GIC.LAMBDA
N = ['S', 'A', 'B', 'C']
T = ['a', 'b', 'd', 'g', 'h']
P = {
    'S': [['A', 'B', 'C'], ['C', 'b', 'b'], ['B', 'a']],
    'A': [['d', 'a'], ['B', 'C']],
    'B': [['g'], [LAMBDA]],
    'C': [['h'], [LAMBDA]]
}

G = GIC (N, T, P)

G.FirstAll()

{'S': ['', 'a', 'b', 'd', 'g', 'h'],
 'A': ['', 'd', 'g', 'h'],
 'B': ['', 'g'],
 'C': ['', 'h']}

In [4]:
# Exemplu 2 https://www.geeksforgeeks.org/follow-set-in-syntax-analysis/?ref=lbp

LAMBDA = GIC.LAMBDA
N = ['S', 'B', 'C', 'D', 'E', 'F']
T = ['a', 'b', 'c', 'f', 'g', 'h']
P = {
    'S': [['a', 'B', 'D', 'h']],
    'B': [['c', 'C']],
    'C': [['b', 'C'], [LAMBDA]],
    'D': [['E', 'F']],
    'E': [['g'], [LAMBDA]],
    'F': [['f'], [LAMBDA]]
}

G = GIC (N, T, P)
G.FollowAll()

Iteration 1
{'B': array(['f', 'g', 'h'], dtype='<U32'),
 'C': array(['f', 'g', 'h'], dtype='<U32'),
 'D': array(['h'], dtype='<U32'),
 'E': array(['f', 'h'], dtype='<U32'),
 'F': array(['h'], dtype='<U32'),
 'S': ['#']}



In [5]:
# Exemplu 3 https://www.geeksforgeeks.org/follow-set-in-syntax-analysis/?ref=lbp

LAMBDA = GIC.LAMBDA
N = ['S', 'A', 'B', 'C']
T = ['a', 'b', 'd', 'g', 'h']
P = {
    'S': [['A','C','B'], ['C','b','b'], ['B','a']],
    'A': [['d','a'], ['B','C']],
    'B': [['g'], [LAMBDA]],
    'C': [['h'], [LAMBDA]],
}

G = GIC (N, T, P)
G.FollowAll()

Iteration 1
{'A': array(['#', 'g', 'h'], dtype='<U32'),
 'B': array(['#', 'a', 'g', 'h'], dtype='<U32'),
 'C': array(['#', 'b', 'g', 'h'], dtype='<U32'),
 'S': ['#']}



In [6]:
# Seminar 4: Ex1. b)

LAMBDA = GIC.LAMBDA
LAMBDAF = GIC.LAMBDAF
N = ['S', 'A', 'B']
T = ['a', 'b', 'c']
P = {
    'S': [['a', 'S', 'B'], ['a', 'A', 'c']],
    'A': [['b', 'A', 'c'], [LAMBDA]],
    'B': [['c', 'S', 'c'], ['b']]
}

G = GIC (N, T, P)
G.FollowAll(k=2)

Iteration 1
{'A': array(['c#', 'cb', 'cc'], dtype='<U32'),
 'B': array(['#', 'b#', 'ca'], dtype='<U32'),
 'S': array(['#', 'b#', 'c#', 'ca', 'cb', 'cc'], dtype='<U2')}

Iteration 2
{'A': array(['c#', 'cb', 'cc'], dtype='<U32'),
 'B': array(['#', 'b#', 'bb', 'bc', 'c#', 'ca', 'cb', 'cc'], dtype='<U32'),
 'S': array(['#', 'b#', 'bb', 'bc', 'c#', 'ca', 'cb', 'cc'], dtype='<U2')}



In [9]:
# Eliminare recurenta stanga

LAMBDA = GIC.LAMBDA
N = ['S', 'A']
T = ['a', 'b', 'c']
P = {
    'S': [['S', 'b', 'S'], ['a', 'A']],
    'A': [['A', 'c'], [LAMBDA]]
}
G = GIC (N, T, P)

G.ElimRecSt()
pprint(G.P)

G.FollowAll(k=2)

{'A': [['c', "A'"]],
 "A'": [['', "A'"], ['']],
 'S': [['b', 'S', "S'"]],
 "S'": [['a', 'A', "S'"], ['']]}
Iteration 1
{'A': array(['#', 'ac'], dtype='<U32'),
 "A'": [],
 'S': array(['#', 'ac'], dtype='<U2'),
 "S'": array(['#', 'ac'], dtype='<U32')}

Iteration 2
{'A': array(['#', 'ac'], dtype='<U32'),
 "A'": array(['#', 'ac'], dtype='<U32'),
 'S': array(['#', 'ac'], dtype='<U2'),
 "S'": array(['#', 'ac'], dtype='<U32')}



In [8]:
# Factorizare stanga

LAMBDA = GIC.LAMBDA
N = ['S', 'A']
T = ['a', 'b', 'c']
P = {
    'S': [['a', 'A', 'b', 'S'], ['a', 'A'], ['b', 'c'], ['b', 'c', 'A']],
    'A': [['b', 'A', 'c'], [LAMBDA]]
}
G = GIC (N, T, P)

G.FactSt()
pprint(G.P)
G.N

{'A': [['b', 'A', 'c'], ['']],
 'S': [['b', 'c', "S''"], ['a', 'A', "S'"]],
 "S'": [['b', 'S'], ['']],
 "S''": [[''], ['A']]}


['S', 'A', "S'", "S''"]